In [132]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig
from accelerate import init_empty_weights
import json

# 모델 로드
# model_cache_fp = "/Users/songhak/.cache/huggingface/hub/models--yanolja--EEVE-Korean-Instruct-2.8B-v1.0/snapshots/482db2d0ba911253d09342c34d0e42ac871bfea3"

model_id = "Qwen/Qwen2.5-7B"
config = AutoConfig.from_pretrained(model_id)

tokenizer = AutoTokenizer.from_pretrained(model_id)
print("Model weight are being loaded...")
with init_empty_weights():
    model = AutoModelForCausalLM.from_config(config)
# model = AutoModelForCausalLM.from_pretrained(model_cache_fp)

config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Model weight are being loaded...


In [134]:
# gradient zero masking
from random import sample
import re
vocab = tokenizer.get_vocab()  # {token: id}
korean_ids = []
examples = []
for tok, idx in vocab.items():
    # 아주 러프하게: 토큰 문자열에 한글 문자가 하나라도 있으면 한국어 토큰으로 간주
    decoded = tokenizer.decode([idx])
    if any(re.findall("[ㄱ-ㅣ,가-힣]", decoded)):
        korean_ids.append(idx)
        examples.append("".join(decoded))

korean_ids = sorted(set(korean_ids))
emb = model.get_input_embeddings()  # nn.Embedding

# check ratio
print(f"{len(korean_ids) / len(vocab) * 100:0.2f}% are KOR tokens: Samples ({sample(examples, 5)})")

vocab_size, dim = emb.weight.shape
mask = torch.zeros_like(emb.weight, dtype=torch.bool)
mask[korean_ids] = True            # 한국어 토큰 row만 True

def grad_mask_hook(grad):
    # grad: [vocab_size, dim]
    # 한국어 토큰이 아닌 row의 gradient 0으로
    return grad * mask.to(grad.device)

emb.weight.register_hook(grad_mask_hook)

# how to test simply?

2.46% are KOR tokens: Samples ([' 않고', ' 수정', ' 일', ' 있음', ' 전문'])


In [135]:
# split embedding
class SplitEmbedding(nn.Module):
    def __init__(self, base_weight: torch.Tensor, num_new_tokens: int):
        super().__init__()
        # base_weight: [base_vocab_size, dim] (이미 학습된 weight)
        self.base_vocab_size, self.dim = base_weight.shape

        # 기존 embedding은 gradient 없는 buffer로 등록
        self.register_buffer("embed_base", base_weight)  # [V_base, D]

        # 새 한국어 토큰 부분만 trainable embedding
        self.embed_new = nn.Embedding(num_new_tokens, self.dim)
        nn.init.normal_(self.embed_new.weight, mean=0.0, std=0.02)

    def forward(self, input_ids: torch.LongTensor):
        # input_ids: [B, T]
        device = input_ids.device
        out = torch.empty(
            (*input_ids.shape, self.dim),
            device=device,
            dtype=self.embed_base.dtype,
        )

        base_mask = input_ids < self.base_vocab_size
        new_mask = ~base_mask

        if base_mask.any():
            base_ids = input_ids[base_mask]      # 1D tensor
            out[base_mask] = self.embed_base[base_ids]

        if new_mask.any():
            new_ids = input_ids[new_mask] - self.base_vocab_size
            out[new_mask] = self.embed_new(new_ids)

        return out

In [138]:
# new model to merge
print(config)
tokenizer = AutoTokenizer.from_pretrained(model_id)
print("Model weight are being loaded...")

with init_empty_weights():
    model = AutoModelForCausalLM.from_config(config)


def to_scalable_params(x: int) -> str:
    if x > 1000 ** 3:
        return f"{x / 1000**3:.2f}B"
    return f"{x / 1000**2:.2f}M"

def to_scalable_bytes(x: int) -> str:
    if x > 1024 ** 3:
        return f"{x / 1024**3:.2f}GiB"
    return f"{x / 1024**2:.2f}MiB"


def count_params(module: nn.Module) -> tuple[int, int]:
    total = 0
    trainable = 0
    for p in module.parameters():
        n = p.numel()
        total += n
        if p.requires_grad:
            trainable += n

    return to_scalable_params(total), to_scalable_params(trainable)

def module_param_stats(module: torch.nn.Module) -> list[dict]:
    """
    leaf 모듈(자식 모듈이 없는 모듈) 단위로
    파라미터 개수 / 메모리 합을 계산.
    """
    rows: list[dict] = []

    for module_name, module in module.named_modules():
        # 자식이 있는 모듈은 스킵하고 leaf만 본다.
        if any(True for _ in module.children()):
            continue

        num_params = 0
        num_trainable = 0
        num_bytes = 0
        for name, p in module.named_parameters(recurse=False):
            n = p.numel()
            num_params += n
            if p.requires_grad:
                num_trainable += n
            num_bytes += n * p.element_size()

        if num_params == 0:
            continue

        rows.append(
            {
                "name": module_name or "(root)",
                "num_params": num_params,
                "num_trainable": num_trainable,
                "num_bytes": num_bytes,
            }
        )

    # 파라미터 개수 기준 내림차순 정렬
    rows.sort(key=lambda r: r["num_params"], reverse=True)
    return rows


def group_by_name_prefix(
    rows: list[dict],
    prefixes: dict[str, list[str]],
) -> dict[str, dict]:
    """
    이름 prefix로 대략적인 그룹을 만들어 합산.
    예: embedding, transformer block, lm_head 등
    """
    stats: dict[str, dict] = {}
    for group_name in prefixes.keys():
        stats[group_name] = {
            "num_params": 0,
            "num_trainable": 0,
            "num_bytes": 0,
        }
    stats["(others)"] = {"num_params": 0, "num_trainable": 0, "num_bytes": 0}

    for row in rows:
        name = row["name"]
        matched_group = None
        for group_name, pfx_list in prefixes.items():
            if any(name.startswith(pfx) for pfx in pfx_list):
                matched_group = group_name
                break
        if matched_group is None:
            matched_group = "(others)"

        for key in ["num_params", "num_trainable", "num_bytes"]:
            stats[matched_group][key] += row[key]
    for group_name in stats.keys():
        for key in ["num_params", "num_trainable"]:
            stats[group_name][key] = to_scalable_params(stats[group_name][key])
        stats[group_name]["num_bytes"] = to_scalable_bytes(stats[group_name]["num_bytes"])

    return stats

# print(count_params(model))
rows = module_param_stats(model)
result = group_by_name_prefix(rows, {"embed": ["model.embed", "model.rotary_emb"], "fc": ["lm_head"], "layers": ["model.layers"]})
print(json.dumps(result, indent=2))


# N개 토큰이 추가되었다고 가정
assumed_added_num = 50000
base_vocab_size = vocab_size - assumed_added_num

LlamaConfig {
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "dtype": "bfloat16",
  "eos_token_id": 128001,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": {
    "factor": 8.0,
    "high_freq_factor": 4.0,
    "low_freq_factor": 1.0,
    "original_max_position_embeddings": 8192,
    "rope_type": "llama3"
  },
  "rope_theta": 500000.0,
  "tie_word_embeddings": false,
  "transformers_version": "4.57.3",
  "use_cache": true,
  "vocab_size": 128256
}

Model weight are being loaded...
{
  "embed": {
    "num_params": "525.34M",
    "num_trainable": "525.34M",
    "num_bytes": "1002.00MiB"
  },
  "fc": {

In [139]:
# 추가된 임베딩 세팅
old_emb = model.get_input_embeddings()  # nn.Embedding
old_weight = old_emb.weight.data.clone()  # [base_vocab_size, dim] 만 쓰고 싶으면 slice

# 혹시 resize_token_embeddings를 이미 호출했다면, base 부분만 잘라야 함
old_weight_base = old_weight[:base_vocab_size]

split_emb = SplitEmbedding(old_weight_base, num_new_tokens=assumed_added_num)
split_emb.to(model.device)  # model과 같은 디바이스로
model.set_input_embeddings(split_emb)


In [140]:
class SplitLMHead(nn.Module):
    def __init__(self, base_weight: torch.Tensor, new_embed: nn.Embedding):
        super().__init__()
        # base_weight: [V_base, D]
        self.vocab_size_base, self.dim = base_weight.shape

        # base 쪽은 buffer (고정)
        # [D, V_base] 로 transpose 해두면 matmul에 편함
        self.register_buffer("lm_base", base_weight.T)

        # 새 토큰 부분은 input embedding의 embed_new와 tie 하고 싶다면:
        self.embed_new = new_embed  # nn.Embedding
        # 별도의 Linear를 둘 수도 있고, F.linear로 weight 공유만 할 수도 있음

    def forward(self, hidden_states):
        # hidden_states: [B, T, D]
        B, T, D = hidden_states.shape

        # base logits
        logits_base = hidden_states @ self.lm_base  # [B, T, V_base]

        # new token logits: F.linear(hidden, W_new^T)
        logits_new = torch.matmul(
            hidden_states,
            self.embed_new.weight.T,  # [D, V_new]
        )  # [B, T, V_new]

        # 최종 logits concat
        logits = torch.cat([logits_base, logits_new], dim=-1)
        return logits


In [141]:
# shared weight
split_lm = SplitLMHead(
    base_weight=old_weight_base,  # or old_lm_head_weight[:base_vocab_size]
    new_embed=model.get_input_embeddings().embed_new,
)
model.lm_head = split_lm

In [142]:
# turn base params grad off
for p in model.parameters():
    p.requires_grad = False

for p in model.get_input_embeddings().embed_new.parameters():
    p.requires_grad = True
# SplitLMHead는 embed_new만 trainable이면 별도 param은 없음 (또는 있으면 같이 추가)

optimizer = torch.optim.AdamW(
    model.get_input_embeddings().embed_new.parameters(),
    lr=1e-3,
)

In [143]:
# 메모리 차이 계산

import math
from dataclasses import dataclass

DTYPE_BYTES = {
    "fp32": 4,
    "float32": 4,
    "fp16": 2,
    "float16": 2,
    "bf16": 2,
    "bfloat16": 2,
}

def to_mib(x: int) -> float:
    return x / 1024**2

def to_gib(x: int) -> float:
    return x / 1024**3


@dataclass
class EmbeddingMemory:
    vocab: int
    dim: int
    param_bytes: int
    grad_bytes: int
    opt_bytes: int  # optimizer state (Adam m, v 등)
    bytes_per_param: int

    @property
    def total_bytes(self) -> int:
        return self.param_bytes + self.grad_bytes + self.opt_bytes

    def pretty(self, title: str = ""):
        if title:
            print(f"=== {title} ===")
        print(f"- vocab          : {self.vocab}")
        print(f"- dim            : {self.dim}")
        print(f"- bytes/param    : {self.bytes_per_param} B")
        print(f"- param   : {to_gib(self.param_bytes):7.3f} GiB "
              f"({to_mib(self.param_bytes):8.1f} MiB)")
        print(f"- grad    : {to_gib(self.grad_bytes):7.3f} GiB "
              f"({to_mib(self.grad_bytes):8.1f} MiB)")
        print(f"- opt(m,v): {to_gib(self.opt_bytes):7.3f} GiB "
              f"({to_mib(self.opt_bytes):8.1f} MiB)")
        print(f"- TOTAL   : {to_gib(self.total_bytes):7.3f} GiB "
              f"({to_mib(self.total_bytes):8.1f} MiB)")
        print()


def calc_embedding_memory(
    vocab: int,
    dim: int,
    dtype_param: str = "fp16",
    dtype_grad: str = "fp16",
    dtype_moment: str = "fp32",
    optimizer: str = "adam",
) -> EmbeddingMemory:
    """
    embedding 하나에 대해
      - 파라미터 메모리
      - gradient 메모리
      - optimizer state(Adam m,v) 메모리
    를 계산합니다.
    """
    n_params = vocab * dim

    bytes_param = DTYPE_BYTES[dtype_param]
    bytes_grad = DTYPE_BYTES[dtype_grad]
    bytes_moment = DTYPE_BYTES[dtype_moment]

    param_bytes = n_params * bytes_param
    grad_bytes = n_params * bytes_grad

    if optimizer.lower() in ("adam", "adamw"):
        # Adam: 1차 모멘트 m, 2차 모멘트 v 두 개
        opt_bytes = n_params * bytes_moment * 2
    else:
        # 필요 시 다른 optimizer 로직 추가 가능
        opt_bytes = 0

    return EmbeddingMemory(
        vocab=vocab,
        dim=dim,
        param_bytes=param_bytes,
        grad_bytes=grad_bytes,
        opt_bytes=opt_bytes,
        bytes_per_param=bytes_param,
    )


@dataclass
class SplitEmbeddingMemory:
    total_vocab: int
    kor_vocab: int
    dim: int
    baseline: EmbeddingMemory
    split: EmbeddingMemory  # "학습되는 부분(한국어 토큰)"에 대한 grad/state만

    @property
    def split_param_bytes_total(self) -> int:
        """
        Split 구조에서:
        - 파라미터(embedding weight)는 전체 vocab에 대해 존재
          (base 부분은 buffer, kor 부분은 trainable)
        - baseline과 같은 dtype을 사용한다고 가정
        """
        bytes_per_param = self.baseline.bytes_per_param
        return self.total_vocab * self.dim * bytes_per_param

    @property
    def split_total_bytes(self) -> int:
        """
        Split 구조에서의 총 메모리:
        - 파라미터: 전체 vocab
        - grad/state: 한국어 vocab 부분만
        """
        return self.split_param_bytes_total + self.split.grad_bytes + self.split.opt_bytes

    @property
    def saved_bytes(self) -> int:
        return self.baseline.total_bytes - self.split_total_bytes

    @property
    def split_ratio(self) -> float:
        """
        Split 구조 총 메모리 / Baseline 총 메모리
        (즉, 기존 대비 몇 % 메모리만 쓰는지)
        """
        return self.split_total_bytes / self.baseline.total_bytes

    @property
    def saved_ratio(self) -> float:
        """
        절약 비율: 1 - split_ratio
        (기존 대비 얼마나 줄었는지)
        """
        return 1.0 - self.split_ratio

    def pretty(self):
        print("====== Baseline: 전체 vocab 학습 ======")
        self.baseline.pretty("Baseline (all tokens trainable)")

        print("====== Split: 한국어 토큰만 학습 ======")
        print(f"- total_vocab : {self.total_vocab}")
        print(f"- kor_vocab   : {self.kor_vocab}")
        print(f"- dim         : {self.dim}")
        print()

        print(">> 한국어 토큰 부분(trainable embedding)만 기준으로 한 메모리")
        self.split.pretty("Trainable Korean part only")

        print(">> 실제 Split 구조에서의 총 메모리 추정")
        print(f"- 파라미터(전체 vocab) : "
              f"{to_gib(self.split_param_bytes_total):7.3f} GiB "
              f"({to_mib(self.split_param_bytes_total):8.1f} MiB)")
        print(f"- grad(한국어 토큰만) : "
              f"{to_gib(self.split.grad_bytes):7.3f} GiB "
              f"({to_mib(self.split.grad_bytes):8.1f} MiB)")
        print(f"- opt (한국어 토큰만) : "
              f"{to_gib(self.split.opt_bytes):7.3f} GiB "
              f"({to_mib(self.split.opt_bytes):8.1f} MiB)")
        print(f"- TOTAL(SPLIT)        : "
              f"{to_gib(self.split_total_bytes):7.3f} GiB "
              f"({to_mib(self.split_total_bytes):8.1f} MiB)")
        print()

        # 절약량 (절대값)
        print(">> 절감된 메모리 (Baseline - Split)")
        print(f"- saved bytes : {to_gib(self.saved_bytes):7.3f} GiB "
              f"({to_mib(self.saved_bytes):8.1f} MiB)")
        print()

        # 비율 (질문하신 부분)
        print(">> Baseline 대비 Split 메모리 비율")
        print(f"- Split / Baseline : {self.split_ratio * 100:5.2f}%")
        print(f"- Saved            : {self.saved_ratio * 100:5.2f}% 감소")
        print()


In [144]:
# total_vocab = vocab_size
# kor_vocab = assumed_added_num
# dim = old_weight_base.shape[-1]


kor_vocab = assumed_added_num
total_vocab = vocab_size
dim = 1024


dtype_param = "fp32"
dtype_grad = "fp32"
dtype_moment = "fp32"   # Adam 모멘트 fp32 가정
optimizer = "adamw"


baseline = calc_embedding_memory(
    vocab=total_vocab,
    dim=dim,
    dtype_param=dtype_param,
    dtype_grad=dtype_grad,
    dtype_moment=dtype_moment,
    optimizer=optimizer,
)

split_trainable = calc_embedding_memory(
    vocab=kor_vocab,
    dim=dim,
    dtype_param=dtype_param,
    dtype_grad=dtype_grad,
    dtype_moment=dtype_moment,
    optimizer=optimizer,
)

result = SplitEmbeddingMemory(
        total_vocab=total_vocab,
        kor_vocab=kor_vocab,
        dim=dim,
        baseline=baseline,
        split=split_trainable,
    )
result.pretty()

====== Baseline: 전체 vocab 학습 ======
=== Baseline (all tokens trainable) ===
- vocab          : 128256
- dim            : 1024
- bytes/param    : 4 B
- param   :   0.489 GiB (   501.0 MiB)
- grad    :   0.489 GiB (   501.0 MiB)
- opt(m,v):   0.979 GiB (  1002.0 MiB)
- TOTAL   :   1.957 GiB (  2004.0 MiB)

====== Split: 한국어 토큰만 학습 ======
- total_vocab : 128256
- kor_vocab   : 50000
- dim         : 1024

>> 한국어 토큰 부분(trainable embedding)만 기준으로 한 메모리
=== Trainable Korean part only ===
- vocab          : 50000
- dim            : 1024
- bytes/param    : 4 B
- param   :   0.191 GiB (   195.3 MiB)
- grad    :   0.191 GiB (   195.3 MiB)
- opt(m,v):   0.381 GiB (   390.6 MiB)
- TOTAL   :   0.763 GiB (   781.2 MiB)

>> 실제 Split 구조에서의 총 메모리 추정
- 파라미터(전체 vocab) :   0.489 GiB (   501.0 MiB)
- grad(한국어 토큰만) :   0.191 GiB (   195.3 MiB)
- opt (한국어 토큰만) :   0.381 GiB (   390.6 MiB)
- TOTAL(SPLIT)        :   1.061 GiB (  1086.9 MiB)

>> 절감된 메모리 (Baseline - Split)
- saved bytes :   0.896 GiB (   917.1 Mi